# Day 13 - Distress Signal

https://adventofcode.com/2022/day/13

In [157]:
from pathlib import Path
import json
import ipytest, pytest

ipytest.autoconfig(addopts=["-v"])

INPUTS = Path("input.txt").read_text().strip()
SIGNALS = [list(map(json.loads, x.split())) for x in INPUTS.split("\n\n")]


In [158]:
print(">> Example:")
print(f"Signal 1: {SIGNALS[0][0]}")
print(f"Signal 2: {SIGNALS[0][1]}")


>> Example:
Signal 1: [[[[8, 7, 8, 5, 4], 6, [4, 6]]], [6, [[]]], []]
Signal 2: [[[[8, 0, 0, 7, 1], [1], 8]]]


## Part 1

In [159]:
from itertools import zip_longest

SignalType = list[list] | list[int] | int


def is_in_order(left: SignalType, right: SignalType) -> bool:
    # Use the other method to run recursive checks
    result = _is_in_order(left, right)
    # If we got None as a final result, then the items are identical.
    # Tie goes to True, I believe?
    if result is None:
        return True
    return result


def _is_in_order(left: SignalType, right: SignalType) -> bool | None:
    # If the types are a mismatch, wrap the int in list before continuing
    if isinstance(left, int) and not isinstance(right, int):
        left = [left]
    if isinstance(right, int) and not isinstance(left, int):
        right = [right]

    # compare the elements of left and right
    for elem_left, elem_right in zip_longest(left, right):
        # If the left is exhausted first, it will be None.
        # Thus, they are in order
        if elem_left is None:
            return True
        # In the opposite scenario, right was exhausted first.
        # They are not in order
        if elem_right is None:
            return False

        # If either of the types of the left or right elems are lists,
        # we have to recurse into them
        if isinstance(elem_left, list) or isinstance(elem_right, list):
            result = _is_in_order(elem_left, elem_right)
            # Result is None if we don't yet know.
            # Otherwise, we got a determination: exit now
            if result is not None:
                return result
            continue

        # Finally, integer comparisons can be performed.
        if elem_left < elem_right:
            return True
        if elem_right < elem_left:
            return False
    # If we got this far, then the elements we iterated through are the same.
    # Thus, we don't know if they're in order.
    return None


In [160]:
%%ipytest

@pytest.mark.parametrize("signals, expected", [
    ("[1,1,3,1,1]\n[1,1,5,1,1]", True),
    ("[[1],[2,3,4]]\n[[1],4]", True),
    ("[9]\n[[8,7,6]]", False),
    ("[[4,4],4,4]\n[[4,4],4,4,4]", True),
    ("[7,7,7,7]\n[7,7,7]", False),
    ("[]\n[3]", True),
    ("[[[]]]\n[[]]", False),
    ("[1,[2,[3,[4,[5,6,7]]]],8,9]\n[1,[2,[3,[4,[5,6,0]]]],8,9]", False),
])
def test_in_order(signals: str, expected: bool):
    pairs = list(map(json.loads, signals.split('\n')))
    result = is_in_order(*pairs)
    assert result == expected



======================================= test session starts ========================================
platform darwin -- Python 3.10.7, pytest-7.2.0, pluggy-1.0.0 -- /Users/garice/Library/Caches/pypoetry/virtualenvs/griceturrble-advent-of-code-8jQN35Cx-py3.10/bin/python
cachedir: .pytest_cache
rootdir: /Users/garice/dev/gits/personal/advent-of-code/2022/day13
collecting ... collected 8 items

t_1d7056d35eb44c149434eb7149731477.py::test_in_order[[1,1,3,1,1]\n[1,1,5,1,1]-True] PASSED   [ 12%]
t_1d7056d35eb44c149434eb7149731477.py::test_in_order[[[1],[2,3,4]]\n[[1],4]-True] PASSED     [ 25%]
t_1d7056d35eb44c149434eb7149731477.py::test_in_order[[9]\n[[8,7,6]]-False] PASSED            [ 37%]
t_1d7056d35eb44c149434eb7149731477.py::test_in_order[[[4,4],4,4]\n[[4,4],4,4,4]-True] PASSED [ 50%]
t_1d7056d35eb44c149434eb7149731477.py::test_in_order[[7,7,7,7]\n[7,7,7]-False] PASSED        [ 62%]
t_1d7056d35eb44c149434eb7149731477.py::test_in_order[[]\n[3]-True] PASSED                    [ 75%]
t_1d7

In [161]:
results = []
for left, right in SIGNALS:
    results.append(is_in_order(left, right))


In [162]:
answer = sum(idx for idx, val in enumerate(results, start=1) if val)
print(f"{answer=}")


answer=6240


## Part 2

So, we want to sort these signals, inject a couple new ones, and see where they are.

Sounds like we need to make a data type that knows how to compare itself (`__lt__`, `__gt__`, and `__eq__` magic methods). Then we'll just be able to use `sorted()` to get a sorted list of those signals. Easy peasy.

The hangups designing the class below came in producing an iterator with `__iter__` so the original function could work. We also had to go back and move `.split('\n\n')` out of the `INPUTS` assignment and into the `SIGNALS` one, so our original inputs wouldn't be mangled.

Overall, though, very straightforward system. Not even a neat to create our own sorting algorithm: just let Python work it out by telling it how to compare them to each other.

In [163]:
class Signal:
    def __init__(self, data: str):
        self.data = data
        self.signal_list = json.loads(data)

    def __str__(self) -> str:
        return self.data

    def __repr__(self) -> str:
        return f"<Signal(data='{self.data}')>"

    def __lt__(self, other):
        return is_in_order(self, other)

    def __gt__(self, other):
        return is_in_order(other, self)

    def __iter__(self):
        return iter(self.signal_list)

    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return other.data == self.data
        if isinstance(other, str):
            return other == self.data
        if isinstance(other, list):
            return other == self.signal_list
        return False


def solve_part2(inputs: str) -> int:
    all_signals = [Signal(x) for x in (inputs.split("\n") + ["[[2]]", "[[6]]"]) if x]
    sorted_signals = sorted(all_signals)
    keys = [
        idx
        for idx, val in enumerate(sorted_signals, start=1)
        if val in ("[[2]]", "[[6]]")
    ]
    return keys[0] * keys[1]


I got to the end of this and produced an answer that worked, but testing is still important! Let's try out the sample data from before.

In [164]:
%%ipytest

sample = """[1,1,3,1,1]
[1,1,5,1,1]

[[1],[2,3,4]]
[[1],4]

[9]
[[8,7,6]]

[[4,4],4,4]
[[4,4],4,4,4]

[7,7,7,7]
[7,7,7]

[]
[3]

[[[]]]
[[]]

[1,[2,[3,[4,[5,6,7]]]],8,9]
[1,[2,[3,[4,[5,6,0]]]],8,9]"""

def test_sorting():
    answer = solve_part2(sample)
    assert answer == 140

======================================= test session starts ========================================
platform darwin -- Python 3.10.7, pytest-7.2.0, pluggy-1.0.0 -- /Users/garice/Library/Caches/pypoetry/virtualenvs/griceturrble-advent-of-code-8jQN35Cx-py3.10/bin/python
cachedir: .pytest_cache
rootdir: /Users/garice/dev/gits/personal/advent-of-code/2022/day13
collecting ... collected 1 item

t_1d7056d35eb44c149434eb7149731477.py::test_sorting PASSED                                   [100%]

======================================== 1 passed in 0.01s =========================================


Looking good! Now for our final result.

In [165]:
answer = solve_part2(INPUTS)
print(f"{answer=}")


answer=23142
